# Traveling Salesman Problem

A salesman has to visit some clients and then come back home for 
dinner.
Given a list of clients and the distances between each pair 
of them, what is the shortest possible route that visits 
each client exactly once and returns to its home? 


<li> <font color="blue">Decision variables:</font>

For each pair of nodes $i,j\in N$ we define
 
$$x_{ij}=\left\{\begin{array}{ll}
1 & \textrm{if it visits node $j$ right after node $i$}\\
0 & \textrm{otherwise}
\end{array}\right.$$

<li> <font color="blue">Objective:</font>: 

$$\min_{x_{ij}} \sum\nolimits_{i\neq j} c_{ij} x_{ij}$$


<li> <font color="blue">Constraints:</font>: 

Each node must be arrived from exactly one node: 

$$\sum\limits_{i} x_{ij} = 1$$ 

From each node there is a departure to exactly one other node

$$\sum\limits_{j} x_{ij} = 1$$

Add MTZ (Miller-Tucker-Zemlin) set of constraints:
\begin{align*}
&u_{1}=1\\
&2 \leq u_{i} \leq n \quad \forall i \neq 1\\
&u_{i}-u_{j}+1 \leq (n-1)(1-x_{ij})\quad \forall i \neq 1, \forall j\neq 1
\end{align*}
where $u_{i} \quad i=1,\ldots,n$ are extra variables.












In [34]:
%%writefile travel_sales.py

# NodesIn and NodesOut are intialized using the Arcs
from pyomo.environ import *
model = AbstractModel()
model.n = Param(within=NonNegativeIntegers)
model.I = RangeSet(1,model.n)
model.J = RangeSet(1,model.n)




model.x = Var(model.I, model.J, domain=NonNegativeReals,bounds=(0,1))
model.Cost = Param(model.I, model.J)


#def Obj_rule(model):
#	return summation(model.Cost,model.x)
#model.Obj = Objective(rule=Obj_rule, sense=minimize)

def Obj_rule(model):
	return sum(sum(model.Cost[i,j]*model.x[i,j] for i in model.I if (i!=j)) for j in model.J)
model.Obj = Objective(rule=Obj_rule, sense=minimize)

def in_constraint_rule(model, i): 
    return sum(model.x[i,j] for j in model.J if (j!=i))== 1
model.in_Constraint = Constraint(model.I, rule=in_constraint_rule)

def out_constraint_rule(model, j): 
    return sum(model.x[i,j] for i in model.I if (i!=j))== 1
model.out_Constraint = Constraint(model.J, rule=out_constraint_rule)

#constraints to avoid cycles:
model.u = Var(model.I,domain=NonNegativeReals)

def u1_cons(model,i):
    if i==1:
        return model.u[i]==1
    else:
        return model.u[i]>=2
model.u1_constraint=Constraint(model.I,rule=u1_cons)

def u2_cons(model,i):
    return model.u[i]<=model.n
model.u2_constraint=Constraint(model.I,rule=u2_cons)

def u3_cons(model,i,j):
    if (i!=1 and j!=1):
        return model.u[i]-model.u[j] + 1 <= (model.n-1)*(1-model.x[i,j])
    else:
        return Constraint.Skip
model.u3_constraint=Constraint(model.I,model.J,rule=u3_cons)








Overwriting travel_sales.py


In [35]:
%%writefile travel_sales.dat




param n := 28
;




param Cost :	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28 :=
1	0	369	525	540	646	504	207	354	860	142	506	264	584	578	251	473	150	772	702	598	463	492	473	443	242	191	444	423
2	369	0	604	809	958	651	407	332	1172	511	516	228	896	899	563	219	219	1084	1014	970	763	422	794	711	577	460	756	758
3	525	604	0	1022	694	89	318	272	772	555	251	376	496	676	401	436	675	645	614	755	299	217	579	935	703	716	414	726
4	540	809	1022	0	620	918	811	908	1118	562	1140	804	784	468	621	997	590	1027	902	437	778	1046	453	98	409	349	663	296
5	646	958	694	620	0	605	585	795	644	562	939	730	359	152	395	939	796	605	304	159	395	993	257	555	488	633	280	324
6	504	651	89	918	605	0	324	319	683	451	323	423	407	595	297	506	654	556	525	650	210	264	490	831	599	636	325	622
7	207	407	318	811	585	324	0	201	799	244	433	179	511	526	190	388	357	699	641	597	353	339	421	649	397	398	377	515
8	354	332	272	908	795	319	201	0	995	445	232	104	733	736	400	187	444	921	851	807	529	138	631	796	596	545	578	725
9	860	1172	772	1118	644	683	799	995	0	776	1006	944	334	650	609	1153	1010	175	340	738	473	947	676	1064	907	961	455	833
10	142	511	555	562	562	451	244	445	776	0	677	380	500	464	167	615	292	689	618	535	379	583	359	464	153	220	360	334
11	506	516	251	1140	939	323	433	232	1006	677	0	336	730	968	632	313	628	879	821	1039	533	94	863	1029	842	791	648	957
12	264	228	376	804	730	423	179	104	944	380	336	0	668	671	335	209	340	856	786	742	532	242	566	707	506	455	528	660
13	584	896	496	784	359	407	511	733	334	500	730	668	0	316	333	877	734	271	118	404	197	671	342	719	573	685	134	488
14	578	899	676	468	152	595	526	736	650	464	968	671	316	0	336	880	694	562	391	88	352	874	105	403	336	481	237	172
15	251	563	401	621	395	297	190	400	609	167	632	335	333	336	0	544	401	521	451	407	212	538	231	534	302	352	193	325
16	473	219	436	997	939	506	388	187	1153	615	313	209	877	880	544	0	407	1065	995	951	756	219	775	899	715	648	737	869
17	150	219	675	590	796	654	357	444	1010	292	628	340	734	694	401	407	0	922	852	714	613	534	589	492	385	241	594	539
18	772	1084	645	1027	605	556	699	921	175	689	879	856	271	562	521	1065	922	0	337	650	346	820	574	962	805	873	356	731
19	702	1014	614	902	304	525	641	851	340	618	821	786	118	391	451	995	852	337	0	463	315	789	463	835	694	803	252	604
20	598	970	755	437	159	650	597	807	738	535	1039	742	404	88	407	951	714	650	463	0	440	945	176	372	356	501	325	175
21	463	763	299	778	395	210	353	529	473	379	533	532	197	352	212	756	613	346	315	440	0	474	325	713	514	564	115	482
22	492	422	217	1046	993	264	339	138	947	583	94	242	671	874	538	219	534	820	789	945	474	0	769	949	748	697	589	863
23	473	794	579	453	257	490	421	631	676	359	863	566	342	105	231	775	589	574	463	176	325	769	0	388	231	376	210	157
24	443	711	935	98	555	831	649	796	1064	464	1029	707	719	403	534	899	492	962	835	372	713	949	388	0	311	251	598	231
25	242	577	703	409	488	599	397	596	907	153	842	506	573	336	302	715	385	805	694	356	514	748	231	311	0	145	441	181
26	191	460	716	349	633	636	398	545	961	220	791	455	685	481	352	648	241	873	803	501	564	697	376	251	145	0	545	326
27	444	756	414	663	280	325	377	578	455	360	648	528	134	237	193	737	594	356	252	325	115	589	210	598	441	545	0	367
28	423	758	726	296	324	622	515	725	833	334	957	660	488	172	325	869	539	731	604	175	482	863	157	231	181	326	367	0
;


Overwriting travel_sales.dat


In [36]:
!pyomo solve  travel_sales.py  travel_sales.dat --solver=glpk --summary 

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.09] Applying solver
[    0.19] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 3996.85185185
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=784, Index=x_index
        Key      : Lower : Value             : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :              None :     1 : False :  True : NonNegativeReals
          (1, 2) :     0 :               0.0 :     1 : False : False : NonNegativeReals
          (1, 3) :     0 :               0.0 :     1 : False : False : NonNegativeReals
          (1, 4) :     0 :               0.0 :     1 : False : False : NonNegativeReals
          (1, 5) :     0 :               0.0 :     1 : False : False : NonNegativeReals
          (1, 6) :     0 :               0.0 :     1 : False : False 

In [37]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3996.85185185
  Upper bound: 3996.85185185
  Number of objectives: 1
  Number of constraints: 842
  Number of variables: 812
  Number of nonzeros: 3702
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0309998989105
# ----------------------------------------------------------
#   Solution Information
# ------------------